# North Korea provocations database

In [1]:
%load_ext lab_black

In [3]:
import pandas as pd
import geopandas as gpd
import altair as alt
import altair_stiles as altstiles
from altair_saver import save
import numpy as np
import us
import urllib.request, json
import glob
import os
import requests
from bs4 import BeautifulSoup
import os
import time
import matplotlib.pyplot as plt
import squarify
from matplotlib.pyplot import figure

In [4]:
alt.themes.register("stiles", altstiles.theme)
alt.themes.enable("stiles")
figure(figsize=(8, 6), dpi=80)

<Figure size 640x480 with 0 Axes>

<Figure size 640x480 with 0 Axes>

In [5]:
pd.options.display.max_columns = 50
pd.options.display.max_rows = 1000
alt.data_transformers.disable_max_rows()

DataTransformerRegistry.enable('default')

### Download latest version of the database from here

In [6]:
# https://beyondparallel.csis.org/database-north-korean-provocations/

### Import data

In [7]:
src = pd.read_csv("data/raw/Provocations New.csv")

In [8]:
src.columns = (
    src.columns.str.strip()
    .str.lower()
    .str.replace(" ", "_", regex=True)
    .str.replace("(", "", regex=True)
    .str.replace(")", "", regex=True)
    .str.replace("-", "_", regex=True)
)

In [9]:
src["description"].iloc[0]

'Around 10 North Korean military aircraft conducted exercises near the inter-Korean border between 10:30pm and 12:20 am. Some flew as close as 15 miles to the Military Demarcation Line, 7 miles close to the Northern Limit Line and 3 miles close to the no-fly zone agreed to in the 2018 September 19 military agreement.'

### Parse dates

In [10]:
src["date"] = pd.to_datetime(src["date"])
src["year"] = src["date"].dt.year
src["month"] = src["date"].dt.month
src["day"] = src["date"].dt.day
src["day"] = src["date"].dt.day_name()

In [11]:
src["type"] = src["type"].str.replace(" Provocation", "")

In [12]:
len(src)

336

In [13]:
src.tail()

,date,type,event,description,resources,year,month,day
331,1963-07-29,Other,Exchange of Fire at DMZ,A group of at least seven North Korean soldier...,https://goo.gl/8PXjzE; The US Military Experie...,1963,7,Monday
332,1962-11-20,Other,Exchange of Fire at DMZ,NK troops attacked a UN observation post 7 mil...,http://www.cfr.org/content/publications/attach...,1962,11,Tuesday
333,1960-07-30,Other,Exchange of Fire and Sinking of NK Vessel,A South Korean destroyer sinks a North Korean ...,https://goo.gl/NEgR7x The US Military Experien...,1960,7,Saturday
334,1959-06-15,Other,NK Fighter Jets Attack U.S. Navy Plane,Two North Korean jet fighters (MiG) attacked a...,https://goo.gl/UFh93r The US Military Experien...,1959,6,Monday
335,1958-02-16,Other,Hijacking of South Korean Plane,North Korean agents hijacked a South Korean pl...,https://goo.gl/Xn9Wr4 The US Military Experien...,1958,2,Sunday


---

## Aggregate

#### Provocation types

In [14]:
prov_types_src = (
    src.groupby(["type", "year"])
    .agg({"date": "count"})
    .reset_index()
    .rename(columns={"date": "count"})
)

In [15]:
prov_types_src.head(27)

,type,year,count
0,Missile,1984,2
1,Missile,1985,1
2,Missile,1990,2
3,Missile,1991,2
4,Missile,1992,1
5,Missile,1993,2
6,Missile,1994,2
7,Missile,1995,1
8,Missile,1997,1
9,Missile,1998,1


In [16]:
# prov_types["date"] = pd.to_datetime("12/31/" + prov_types["year"].astype(str))

In [17]:
dummy = pd.read_csv(
    "https://docs.google.com/spreadsheets/d/e/2PACX-1vT1tUkO4UkOeOl5_O_j5yEHplUJF1NLC1u55kCWg_1HsbTputy80dZJubTzCdHCKPmTgMS8V9KS6cmM/pub?gid=0&single=true&output=csv"
)

In [18]:
prov_types = pd.merge(prov_types_src, dummy, on=["year", "type"], how="right").fillna(0)

In [19]:
prov_types["year"] = prov_types["year"].astype(str)

In [20]:
prov_types[prov_types["year"] == "2022"]

,type,year,count
62,Missile,2022,27.0
125,Nuclear,2022,0.0
188,Other,2022,7.0


---

## Charts

#### Extended

In [21]:
circles = (
    alt.Chart(prov_types)
    .mark_circle(thickness=2, opacity=0.5)
    .encode(
        alt.X(
            "year:O",
            axis=alt.Axis(
                values=["1960", "1970", "1980", "1990", "2000", "2010", "2022"]
            ),
            title=" ",
        ),
        alt.Y("type:N", title=" "),
        alt.Size(
            "count:Q",
            title="Count",
            scale=alt.Scale(range=[0, 2000]),
            legend=None,
        ),
        alt.Color("type:N", legend=None),
    )
    .properties(
        width=820,
        height=200,
        title="",
    )
)

(circles).configure_legend(symbolType="circle")

/Users/mstiles/.local/share/virtualenvs/notebooks-pUaZ2fM9/lib/python3.9/site-packages/altair/utils/core.py:317: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for col_name, dtype in df.dtypes.iteritems():


alt.Chart(...)

#### Tablet

In [22]:
circles = (
    alt.Chart(prov_types, padding={"top": -30})
    .mark_circle(thickness=2, opacity=0.5)
    .encode(
        alt.X(
            "year:O",
            axis=alt.Axis(
                values=["1960", "1970", "1980", "1990", "2000", "2010", "2022"]
            ),
            title=" ",
        ),
        alt.Y("type:N", title=" "),
        alt.Size(
            "count:Q",
            title="Count",
            scale=alt.Scale(range=[0, 1000]),
            legend=None,
        ),
        alt.Color("type:N", legend=None),
    )
    .properties(
        width=440,
        height=200,
        title="",
    )
)

(circles)

alt.Chart(...)

#### Mobile

In [23]:
circles = (
    alt.Chart(prov_types, padding={"top": -30})
    .mark_circle(thickness=2, opacity=0.5)
    .encode(
        alt.X(
            "year:O",
            axis=alt.Axis(
                values=["1960", "1970", "1980", "1990", "2000", "2010", "2022"]
            ),
            title=" ",
        ),
        alt.Y("type:N", title=" "),
        alt.Size(
            "count:Q",
            title="Count",
            scale=alt.Scale(range=[0, 1000]),
            legend=None,
        ),
        alt.Color("type:N", legend=None),
    )
    .properties(
        width=300,
        height=200,
        title="",
    )
)

(circles)

alt.Chart(...)

#### Wide

In [26]:
prov_types.head()

,type,year,count
0,Missile,1960,0.0
1,Missile,1961,0.0
2,Missile,1962,0.0
3,Missile,1963,0.0
4,Missile,1964,0.0


In [24]:
circles_wide = (
    alt.Chart(prov_types, padding={"top": -30})
    .mark_circle(thickness=2, opacity=0.5)
    .encode(
        alt.X(
            "year:O",
            axis=alt.Axis(
                values=["1960", "1970", "1980", "1990", "2000", "2010", "2022"]
            ),
            title=" ",
        ),
        alt.Y("type:N", title=" "),
        alt.Size(
            "count:Q",
            title="Count",
            scale=alt.Scale(range=[0, 1000]),
            legend=None,
        ),
        alt.Color("type:N", legend=None),
    )
    .properties(
        width=580,
        height=300,
        title="",
    )
)

circles_wide

alt.Chart(...)